# Threshold based mapping


In [5]:
import ee
import geemap
import numpy as np

In [6]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-vikramscience85')

In [21]:
#Define Polygon for clipping purpose

aoi = ee.Geometry.Polygon([[77.26959756556572,26.223227530131734],
[77.3154311532122,26.223227530131734],
[77.3154311532122,26.256947297291266],
[77.26959756556572,26.256947297291266], 
[77.26959756556572,26.223227530131734]])


### Define Functions

In [20]:
# Function to mask pixels with low CS+ QA scores.
def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.5
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)

# Function to add NDVI, time, and constant variables
def addNDVI(image):
    #Return the image with the added bands.
    ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
    return image.addBands([ndvi])

def permWaterMask():
    permWater1 = ee.ImageCollection("ESA/WorldCover/v200")\
            .first()\
            .select('Map')\
            .eq(80)
    permWater2 = ee.ImageCollection("ESA/WorldCover/v100")\
            .first()\
            .select('Map')\
            .eq(80)
    return permWater1.Or(permWater2)

def settleMask():
    settle1 = ee.ImageCollection("ESA/WorldCover/v200")\
            .first()\
            .select('Map')\
            .eq(50)
    settle2 = ee.ImageCollection("ESA/WorldCover/v100")\
            .first()\
            .select('Map')\
            .eq(50)
    return settle1.Or(settle2)

def Prethreshold(img1, img2, imag3):
    bl = img1.lte(0.4).And(img2.lte(0.4).And(img3.lte(0.4)))
    return bl

def Postthreshold(img1, img2, imag3):
    ll = img1.gte(0.4).And(img2.gte(0.4).And(img3.gte(0.4)))
    return ll

def clipFun(img):
    return img.clip(aoi)

### Define the time filter

In [68]:
# This is for previous year
# Define Year
year = '2025'

# Define intervals
jan1fn = ['-01-01', '-01-16']
jan2fn = ['-01-16', '-02-01']
feb1fn = ['-02-01', '-02-16']
feb2fn = ['-02-16', '-03-01']
mar1fn = ['-03-01', '-03-16']
mar2fn = ['-03-16', '-04-01']
print(year+jan1fn[0])
print(year+jan1fn[1])


2025-01-01
2025-01-16


In [69]:
# Define satellite Image collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
filtered = s2\
           .filter(ee.Filter.date(year+jan1fn[0], year+mar2fn[1]))\
           .filter(ee.Filter.bounds(aoi))
# Load the Cloud Score+ collection
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
csPlusBands = csPlus.first().bandNames()

# We need to add Cloud Score + bands to each Sentinel-2
# image in the collection
# This is done using the linkCollection() function
filteredS2WithCs = filtered.linkCollection(csPlus, csPlusBands)

jan1fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+jan1fn[0], year+jan1fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('J1')

jan2fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+jan2fn[0], year+jan2fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('J2')

feb1fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+feb1fn[0], year+feb1fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('F1')

feb2fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+feb2fn[0], year+feb2fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('F2')

mar1fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+mar1fn[0], year+mar1fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('M1')

mar2fnIC = filteredS2WithCs\
    .filter(ee.Filter.date(year+mar2fn[0], year+mar2fn[1]))\
    .map(maskLowQA)\
    .select('B.*')\
    .map(addNDVI)\
    .median()\
    .select('ndvi')\
    .clip(aoi)\
    .rename('M2')


In [70]:
#Identify BL

composite = jan1fnIC.addBands(jan2fnIC).addBands(feb1fnIC).addBands(feb2fnIC).addBands(mar1fnIC).addBands(mar2fnIC)
BL = jan1fnIC.lte(0.4).And(jan2fnIC.lte(0.4)).And(feb1fnIC.lte(0.4)).And(feb2fnIC.lte(0.4)).And(mar1fnIC.lte(0.4)).And(mar2fnIC.lte(0.4))
BL = BL.multiply(1)
BL.getInfo()

{'type': 'Image',
 'bands': [{'id': 'J1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [2, 2],
   'origin': [76, 25],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [71]:
Map = geemap.Map()
Map.addLayer(aoi, {}, 'AOI')
Map.centerObject(aoi, 12)
Map.addLayer(jan1fnIC, {}, "Jan 1FN")
Map.addLayer(jan2fnIC, {}, "Jan 2FN")
Map.addLayer(feb1fnIC, {}, "Feb 1FN")
Map.addLayer(feb2fnIC, {}, "Feb 2FN")
Map.addLayer(mar1fnIC, {}, "Mar 1FN")
Map.addLayer(mar2fnIC, {}, "Mar 2FN")
Map.addLayer(BL, {1:'red'}, "BL")
Map

Map(center=[26.24008841605283, 77.29251435938804], controls=(WidgetControl(options=['position', 'transparent_b…

In [72]:
task = ee.batch.Export.image.toDrive(**{'image':BL,
                                     'description':'BL',
                                     'scale':10,
                                     'folder': "C_BL",
                                     'region':aoi,
                                     'fileNamePrefix':'BL'+'_'+year,
                                     'fileFormat':'GeoTIFF'
                                     })
task.start()